In [3]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModel
from openai import OpenAI
import torch
from tqdm import tqdm
tqdm.pandas()
import os
current_dir = os.getcwd()
os.chdir(current_dir)

In [4]:
df_food = pd.read_csv(r".\商品.txt", sep='\t')
key_word = ['再加一份','再来一碗','加一','加点','加份','加小','另加','多加','我要','整个','有一个','汤里','需要','点菜请点米饭','别忘了米饭','不麻不辣可加两种面','少一点','亲一定','八宝粥八宝粥八宝粥','疙瘩汤汤汤汤汤汤','小米粥南瓜粥','我需要点米饭','很正常的白米饭','小米粥或者玉米粥','加香','加豆','米饭米饭米饭','加鲜嫩小白菜']
wrong_food = df_food[df_food['item_name'].str.startswith(tuple(key_word))]
df_food= df_food.drop(wrong_food.index)

In [6]:
path = "./BAAI"
# 初始化模型和分词器
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)


In [7]:
#转换词向量函数
def cls(text): 
    input = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model(**input).last_hidden_state
    cls_embedding = output[:, 0, :]
    cls_embedding = cls_embedding[0]
    return cls_embedding

In [8]:
df_food['cls'] = df_food['item_name'].progress_apply(cls)

In [9]:
# 存储时将tensor转换为list
df_food['cls'] = df_food['cls'].apply(lambda x: x.tolist())


In [10]:
df_food.to_csv(r"商品-清洗.txt",index=False)